Load the data

In [1]:
import os 
os.getcwd()

'/home/ayan/research/us_location_identifier/src'

In [2]:
import pandas as pd


In [3]:
 
df = pd.read_csv('../data/users_with_locations.csv.gz',compression='gzip')
df_en = pd.read_csv('../data/featureEngineered/users_with_locations_sm.csv.gz',compression='gzip')


In [5]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
# look at all the column types 
df_en.dtypes


reviewer_profile      object
bio_language          object
lang_id1               int64
lang_id2               int64
lang_id3               int64
lang_id4               int64
lang_id5               int64
lang_id6               int64
lang_id7               int64
lang_id8               int64
lang_id9               int64
lang_id10              int64
lang_id11              int64
lang_id12              int64
lang_id13              int64
lang_id14              int64
lang_id15              int64
lang_id16              int64
lang_id17              int64
lang_id18              int64
lang_id19              int64
lang_id20              int64
lang_id21              int64
lang_id22              int64
lang_id23              int64
lang_id24              int64
lang_id25              int64
lang_id26              int64
lang_id27              int64
lang_id28              int64
lang_id29              int64
lang_id30              int64
lang_id31              int64
lang_id32              int64
lang_id33     

In [6]:
# Set higher but reasonable limits
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
df_en['bio_language'].head()

0    en
1    no
2    en
3    en
4    en
Name: bio_language, dtype: object